<a href="https://colab.research.google.com/github/AndreMartins21/Analise-Musical/blob/main/ANALISE_MUSICAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ANÁLISE DE ARTISTAS MUSICAIS

- Fonte: [Kaggle](https://www.kaggle.com/pieca111/music-artists-popularity);


# DATAFRAME PRINCIPAL

In [1]:
# IMPORTAÇÃO DAS BIBLIOTECAS
import pandas as pd
import numpy as np
import plotly.express as px

In [3]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DataSet Kaggle/artists.csv")

df

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (2,4,6) have mixed types.Specify dtype option on import or set low_memory=False.



,mbid,artist_mb,artist_lastfm,country_mb,country_lastfm,tags_mb,tags_lastfm,listeners_lastfm,scrobbles_lastfm,ambiguous_artist
0,cc197bad-dc9c-440d-a5b5-d52ba2e14234,Coldplay,Coldplay,United Kingdom,United Kingdom,rock; pop; alternative rock; british; uk; brit...,rock; alternative; britpop; alternative rock; ...,5381567.0,360111850.0,False
1,a74b1b7f-71a5-4011-9441-d0b5e4122711,Radiohead,Radiohead,United Kingdom,United Kingdom,rock; electronic; alternative rock; british; g...,alternative; alternative rock; rock; indie; el...,4732528.0,499548797.0,False
2,8bfac288-ccc5-448d-9573-c33ea2aa5c30,Red Hot Chili Peppers,Red Hot Chili Peppers,United States,United States,rock; alternative rock; 80s; 90s; rap; metal; ...,rock; alternative rock; alternative; Funk Rock...,4620835.0,293784041.0,False
3,73e5e69d-3554-40d8-8516-00cb38737a1c,Rihanna,Rihanna,United States,Barbados; United States,pop; dance; hip hop; reggae; contemporary r b;...,pop; rnb; female vocalists; dance; Hip-Hop; Ri...,4558193.0,199248986.0,False
4,b95ce3ff-3d05-4e87-9e01-c97b66af13d4,Eminem,Eminem,United States,United States,turkish; rap; american; hip-hop; hip hop; hiph...,rap; Hip-Hop; Eminem; hip hop; pop; american; ...,4517997.0,199507511.0,False
...,...,...,...,...,...,...,...,...,...,...
1466078,1eab523e-98ff-4083-aa34-8922740bc696,정은지,NaN,South Korea,South Korea,NaN,NaN,NaN,NaN,False
1466079,a18f0527-907e-42b0-8268-504966274581,남태현,NaN,South Korea,NaN,NaN,NaN,NaN,NaN,False
1466080,20a57e37-24b5-4301-855b-35076580fb88,헤일로,NaN,South Korea,NaN,NaN,NaN,NaN,NaN,False
1466081,83891a4d-1bf4-4abe-a483-5b3d9d614efa,서현진,NaN,South Korea,South Korea,NaN,NaN,NaN,NaN,False


# PAÍSES COM MAIOR NÚMERO DE PARTICIPANTES NA PESQUISA


In [ ]:
# SEPARANDO UM DATA SET CONTENDO APENAS OS PAÍSES, A FIM DE FACILITAR O DESENVOLVIMENTO DA ANÁLISE

df_paises = df[["country_mb", "country_lastfm"]].copy()

df_paises.dropna(inplace = True)

df_paises["index"] = np.arange(df_paises.shape[0])

df_paises.set_index("index", inplace = True)

df_paises.head()

,country_mb,country_lastfm
index,,
0,United Kingdom,United Kingdom
1,United Kingdom,United Kingdom
2,United States,United States
3,United States,Barbados; United States
4,United States,United States


In [ ]:
# MESCLANDO OS PAÍSES DE AMBAS COLUNAS EM UM DICIONÁRIO E COLOCANDO-OS EM ORDEM DECRESCENTE

from operator import itemgetter

dict_paises = {}

for ind, value in df_paises.iterrows():
  for cont in range(2):
    valor = value[cont] # cont 0: country_mb, cont 1: country_lastfm

    if valor in dict_paises.keys():
      dict_paises[valor] += 1
    
    else:
      dict_paises[valor] = 1


'COLOCANDO EM ORDEM DECRESCENTE'

paises_ordenados = sorted(dict_paises.items(), key = itemgetter(1), reverse= True)[:5]

print("Países colocados em ordem decrescente com sucesso!")

Países colocados em ordem decrescente com sucesso!


In [ ]:
# CRIANDO O DATA FRAME E PLOTANDO O GRÁFICO

df_ordem_paises = pd.DataFrame({
    "PAÍS": [x[0] for x in paises_ordenados],
    "QUANTIA": [x[1] for x in paises_ordenados]
})

fig = px.bar(df_ordem_paises, x = "PAÍS", y = "QUANTIA", title="PAÍSES COM MAIOR NÚMERO DE PARTICIPANTES NA PESQUISA", color = "QUANTIA")
fig.show()
fig.write_html("PLOT PAISES.html")

# DATAFRAME MusicBrainz


In [ ]:
# Separando as colunas de cada dataframe
analise_mb = df[["artist_mb", "country_mb", "tags_mb"]].copy()

# Retirando todos valores NaN / Null
analise_mb.dropna(inplace= True)

# Organizando o index
analise_mb["index"] = np.arange(0, 94955)

analise_mb.set_index("index", inplace= True)

analise_mb.head(2)

,artist_mb,country_mb,tags_mb
index,,,
0,Coldplay,United Kingdom,rock; pop; alternative rock; british; uk; brit...
1,Radiohead,United Kingdom,rock; electronic; alternative rock; british; g...


## **<> 5 PRINCIPAIS ARTISTAS CITADOS - MUSICBRAINZ**

- Para realizar essa análise, primeiramente separei uma variável que receberá um arquivo Series, o qual irá conter os 5 principais artistas e seus respectivos números de aparição;

- Depois disso, criei um novo dataframe contendo, separadamente, o index e a coluna valores do Series;

- Por fim, utilizei a biblioteca *plotly.express* para visualizar em forma de gráficos quais foram os top cinco artistas.


In [ ]:
artistasmb = analise_mb["artist_mb"].value_counts().head()

df_artistas_mb = pd.DataFrame({
    "ARTISTAS": artistasmb.index,
    "N° DE APARIÇÃO": artistasmb.values
}) 

fig = px.bar(df_artistas_mb, x= "ARTISTAS", y = "N° DE APARIÇÃO", color = "N° DE APARIÇÃO", title= "TOP 5 ARTISTAS - MUSICBRAINZ")
fig.show()
fig.write_html("TOP 5 ARTISTAS - MusicBrainz.html")

## **<> 5 PRINCIPAIS GÊNEROS MUSICAIS - MUSICBRAINZ**


Analisando o dataframe "analise_mb", percebe-se que na coluna ["tags_mb"] os gêneros musicais de cada artista estão separados entre ";".
Para resolver todo esse impasse e selecionar os cinco gêneros musicais mais populares, utilizei o seguinte algoritmo:

***1° -> Separar os estilos musicais de cada artista em uma lista;***

***2° -> Criar um dicionário contendo TODOS os gêneros musicais e a quantidade de repetições;***

***3° -> Colocar em ordem descrente os valores e criar o gráfico para tal.***



In [ ]:
# 1° -> Separar os estilos musicais de cada artista em uma lista

try:
  analise_mb["tags_mb"] = analise_mb["tags_mb"].apply(lambda x: x.split(';'))
except:
  print("!! Os valores da coluna 'tags_mb' já estão organizada em listas!")
else:
  print("Gêneros separados em listas com sucesso!")

Gêneros separados em listas com sucesso!


In [ ]:
# 2° -> Criar um dicinário contendo TODOS os gêneros musicais e a quantidade de repetições

from operator import itemgetter

dict_tags = {}

for i, value in analise_mb.iterrows():
  for genero in value[2]:
    genero = genero.strip()

    if genero in dict_tags.keys():
      dict_tags[genero] += 1
    else:
      dict_tags[genero] = 1

print("Todos os estilos musicais foram alocados no dicionário!")

Todos os estilos musicais foram alocados no dicionário!


In [ ]:
# 3° -> Colocar em ordem descrente os valores e criar o gráfico para tal.


tags_em_ordem = sorted(dict_tags.items(), key=itemgetter(1), reverse=True)[:5]


df_musical_mb = pd.DataFrame({
    "Gênero Musical": [x[0] for x in tags_em_ordem],
    "Quantidade": [z[1] for z in tags_em_ordem]
})

fig = px.bar(df_musical_mb, x = "Gênero Musical", y = "Quantidade", color = "Quantidade", title="5 GÊNEROS MUSICAIS MAIS CITADOS - MUSICBRAINZ")
fig.show()
fig.write_html("RANK GENEROS MUSICAIS- MUSICBRAINZ.html")

# DATAFRAME Last.fm 


In [ ]:
# CRIANDO O DATAFRAME SOMENTE COM AS COLUNAS DA LASTFM E RETIRANDO VALORES NA
dflast = df[["artist_lastfm", "country_lastfm", "tags_lastfm"]].copy()

dflast.dropna(inplace= True)

dflast["index"] = np.arange(204418)

dflast.set_index("index", inplace= True)

dflast.head(3)

,artist_lastfm,country_lastfm,tags_lastfm
index,,,
0,Coldplay,United Kingdom,rock; alternative; britpop; alternative rock; ...
1,Radiohead,United Kingdom,alternative; alternative rock; rock; indie; el...
2,Red Hot Chili Peppers,United States,rock; alternative rock; alternative; Funk Rock...


## **<> 5 PRINCIPAIS ARTISTAS CITADOS - LASTFM**

Mesma tática utiliza no MusicBrainz.

In [ ]:
artistaslast = df["artist_lastfm"].value_counts().head()

df_artistas_last = pd.DataFrame({
    "ARTISTAS": artistaslast.index,
    "N° DE APARIÇÃO": artistaslast.values
})

fig = px.bar(df_artistas_last, x = "ARTISTAS", y = "N° DE APARIÇÃO", title = "TOP 5 ARTISTAS - LASTFM", color = "N° DE APARIÇÃO")
fig.show()
fig.write_html("TOP 5 ARTISTAS - LASTFM.html")

## **<> 5 PRINCIPAIS GÊNEROS MUSICAIS - LASTFM**

Mesma estratégia de algoritmo utilizada para desvendar os gêneros do MUSICBRAINZ


In [ ]:
# 1° passo

try:
  dflast["tags_lastfm"] = dflast["tags_lastfm"].apply(lambda x: x.split(';'))
except:
  print("!! Os valores da coluna 'tags_mb' já estão organizada em listas!")
else:
  print("Gêneros separados em listas com sucesso!")
  

Gêneros separados em listas com sucesso!


In [ ]:
# 2° passo

dict_last = {}

for i, v in dflast.iterrows():
  for genre in v[2]:
    genre = genre.strip()
    if genre in dict_last.keys():
      dict_last[genre] += 1
    else:
      dict_last[genre] = 1

print("Todos os estilos musicais foram alocados no dicionário!")

Todos os estilos musicais foram alocados no dicionário!


In [ ]:
# 3° passo

from operator import itemgetter

tags_em_ordem_lastfm = sorted(dict_last.items(), key=itemgetter(1), reverse= True)[:5]

df_musical_last = pd.DataFrame({
    "Gênero Musical": [x[0] for x in tags_em_ordem_lastfm],
    "Quantidade": [y[1] for y in tags_em_ordem_lastfm]
})

fig = px.bar(df_musical_last, x = "Gênero Musical", y = "Quantidade", title= "5 GÊNEROS MUSICAIS MAIS CITADOS - LASTFM", color= "Quantidade")
fig.show()
fig.write_html("RANK GENRE- LSATFM.html")

# TOP 5 GÊNEROS MUSICAIS MUSICBRAINZ + LASTFM

In [ ]:
mescla_dicts = tags_em_ordem_lastfm + tags_em_ordem
mescla_dicts = sorted(mescla_dicts, key= itemgetter(1), reverse= True)

dict_genre = {}

for a in mescla_dicts:
  nome = a[0]
  if a[0] in dict_genre.keys():
    dict_genre[nome] += a[1]
  else:
    dict_genre[nome] = a[1]

print("Todas quantidades de gêneros idênticas foram somadas e armazenadas no dicionário com sucesso!")

Todas quantidades de gêneros idênticas foram somadas e armazenadas no dicionário com sucesso!


In [ ]:
cont = 0
genero = []
quantia = []

for key, value in dict_genre.items():
  if cont == 5:
    break
  genero.append(key)
  quantia.append(value)
  cont += 1

df_genre = pd.DataFrame({
    "GÊNERO MUSICAL": genero,
    "QUANTIDADE": quantia
})

fig = px.bar(df_genre, x = "GÊNERO MUSICAL", y = "QUANTIDADE", color = "QUANTIDADE", title = "5 GÊNEROS MUSICAIS MAIS CITADOS DO DATASET")
fig.show()